In [2]:
from os import sys
sys.path.append("../../meshAfterParty/")
import trimesh
import skeleton_utils as sk
import neuron
from importlib import reload

In [6]:
segment_id = 12345
current_mesh_file = "/notebooks/neuron_mesh_tools/test_neurons/multi_soma_example.off"
description="2_somas_with_merge"

double_soma_neuron = trimesh.load_mesh(current_mesh_file)
sk.graph_skeleton_and_mesh(other_meshes = [double_soma_neuron])

In [7]:
double_neuron = neuron.Neuron(
    mesh=double_soma_neuron,
    segment_id=segment_id,
    description=description
                         )

--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....


--- 0) Total time for preprocessing: 1103.401335477829




--- 1) Finished unpacking preprocessed materials: 1.1444091796875e-05
total_edges = [['S0', 'L0'], ['S0', 'L1'], ['S0', 'L2'], ['S0', 'L3'], ['S0', 'L5'], ['S0', 'L10'], ['S1', 'L1'], ['S1', 'L4'], ['S1', 'L6'], ['S1', 'L7'], ['S1', 'L8'], ['S1', 'L9']]
--- 2) Finished creating neuron connectivity graph: 0.0001461505889892578
Having to generate soma_meshes_face_idx because none in preprocessed data
--- 3a) Finshed generating soma_meshes_face_idx: 1.1001591682434082
Using Poisson Surface Reconstruction to make mesh watertight
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 5268
xvfb-run -n 5268 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/neuron_265710.off -o /notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/neuron_265710_poisson.off -s /notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/poisson_4450

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/neuron_265710.off
removed temporary output file: /notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/neuron_265710_poisson.off
/notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/poisson_445005.mls is being deleted....
Using Poisson Surface Reconstruction to make mesh watertight
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 4316
xvfb-run -n 4316 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/neuron_285757.off -o /notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/neuron_285757_poisson.off -s /notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/poisson_182996.mls


face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/neuron_285757.off
removed temporary output file: /notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/neuron_285757_poisson.off
/notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/poisson_182996.mls is being deleted....
--- 3) Finshed generating soma objects and adding them to concept graph: 18.19528865814209
--- 4a) Finshed generating curr_limb_meshes_face_idx: 3.8176753520965576
checking and resolving cycles
No cycles to fix
checking and resolving cycles
No cycles to fix
checking and resolving cycles
There are loops to resolve and 'no_cycles' parameters set requires us to fix eliminate them
Using mesh_widths for resolving loops
checking and resolving cycles
No cycles to fix
checking and resolving cycles
No cycles to fix
checking and resolving cycles
No cycles to fix
checking and resolving cycles
No cycles to fix
checking and resolving cycles
There are lo

# Save off the Neuron

In [9]:
import system_utils as su
su = reload(su)
double_neuron.save_neuron_object()

No filename/location given so creating own
Saving Object at: /notebooks/neuron_mesh_tools/meshAfterParty/development/12345_2_somas_with_merge.pkl


In [18]:
neuron = reload(neuron)
updated_neuron = neuron.Neuron(double_neuron,minimal_output=True)

Processing Neuorn in minimal output mode...please wait
Total time for neuron instance creation = 24.358986854553223


# Trying to get smaller saving of neuron

In [28]:
su = reload(su)
su.get_file_size("/notebooks/neuron_mesh_tools/meshAfterParty/development/12345_2_somas_with_merge.pkl")/100000000

10.32249217

# Getting Merge Errors 

In [23]:
"""
For every limb
1) Make a copy of the concept_network
2) Get all of the branches that are below a certain threshold
3) Delete these branches from the network
4) Get all of the branch pieces that are not a part of the component with the starting coordinate
5) collect a list of those branch names (will use to visualize later)

To visualize:
For each limb:
1) collect the meshes of all of the branches in the merge list
2) Add to total list to view

"""
import networkx as nx
import matplotlib.pyplot as plt
import networkx_utils as xu
import numpy as np
import numpy_utils as nu
nu = reload(nu)
from tqdm.notebook import tqdm

import neuron_visualizations as nviz
nviz = reload(nviz)



current_neuron = updated_neuron
merge_width_threshold = 400

limb_list = current_neuron.get_limb_node_names()

limb_merge_branches_list = dict()
total_merge_meshes = []

for curr_limb in tqdm(limb_list):

    #1) Get all of the branches that are below a certain threshold
    curr_concept_network = current_neuron.concept_network.nodes[curr_limb]["data"].concept_network.copy()

    #nx.draw(curr_concept_network,with_labels=True)

    #2) Get all of the branches that are below a certain threshold
    below_threshold_nodes = [k for k in curr_concept_network.nodes() if curr_concept_network.nodes[k]["data"].width < merge_width_threshold]

    #visualize the branch and the parts to remove
    curr_limb_mesh = current_neuron.concept_network.nodes[curr_limb]["data"].mesh
    below_threshold_meshes = [curr_concept_network.nodes[k]["data"].mesh for k in below_threshold_nodes]
    # sk.graph_skeleton_and_mesh(main_mesh_verts=curr_limb_mesh.vertices,
    #                           main_mesh_faces=curr_limb_mesh.faces,
    #                           other_meshes=below_threshold_meshes,
    #                           other_meshes_colors="red")

    #3) Delete these branches from the network
    curr_concept_network.remove_nodes_from(below_threshold_nodes)
    seperated_components = list(nx.connected_components(curr_concept_network))

    #3b) Make sure that the original concept graph had a starting component
    try:
        xu.get_starting_node(curr_concept_network,attribute_for_start="starting_coordinate")
    except:
        raise Exception(f"Original {curr_limb} starting graph did not have a starting_coordinate")

    subcomponent_with_starting_node = -1
    non_starter_components = []
    for j,sp_comp in enumerate(seperated_components):
        curr_subgraph = curr_concept_network.subgraph(sp_comp)
        try: 
            starting_component = xu.get_starting_node(curr_subgraph)
            subcomponent_with_starting_node = j
        except:
            non_starter_components.append(list(sp_comp))

    #print(f"subcomponent_with_starting_node={subcomponent_with_starting_node}")

    if subcomponent_with_starting_node == -1:
        raise Exception("Starting Node was below threshold")
    
    total_merge_pieces_leftover = nu.concatenate_lists(non_starter_components)
        
    total_merge_pieces = nu.concatenate_lists([below_threshold_nodes,total_merge_pieces_leftover])

    merge_meshes = [current_neuron.concept_network.nodes[curr_limb]["data"].concept_network.nodes[k]["data"].mesh for k in total_merge_pieces]

#     sk.graph_skeleton_and_mesh(main_mesh_verts=curr_limb_mesh.vertices,
#                               main_mesh_faces=curr_limb_mesh.faces,
#                               other_meshes=merge_meshes,
#                               other_meshes_colors="red")
    
    limb_merge_branches_list[curr_limb] = total_merge_pieces
    
limb_merge_branches_list
nviz.plot_branch_pieces(current_neuron.concept_network,
                       node_to_branch_dict=limb_merge_branches_list,
                      background_mesh=current_neuron.mesh,
                       mesh_alpha = 0.8,
                       html_path=f"{current_neuron.segment_id}_{current_neuron.description}_merge_segments.html")